In [1]:
import pandas as pd
import numpy as np
import dgl.nn as dglnn
from dgl import from_networkx
import torch.nn as nn
import torch as th
import torch.nn.functional as F
import dgl.function as fn
from dgl.data.utils import load_graphs
import networkx as nx
import pandas as pd
import socket
import struct
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import networkx
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import *
from torch_geometric.loader import NeighborSampler, NeighborLoader
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, ResGatedGraphConv, GATv2Conv, SAGEConv, GENConv, DeepGCNLayer, PairNorm, GINConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch.nn.functional as F
from imblearn.under_sampling import RandomUnderSampler
pd.options.mode.use_inf_as_na = True
from collections import Counter
from sklearn.feature_selection import SelectFromModel
import torch.nn as nn
import time
import pickle
from torch.nn import LayerNorm, Linear, ReLU
from torch_scatter import scatter
from tqdm import tqdm
from torch_geometric.loader import RandomNodeSampler
import math
import copy
from sklearn.metrics import f1_score
from torch.optim import lr_scheduler
from sklearn.manifold import TSNE

In [3]:
class GATLayer(nn.Module):
    def __init__(self, in_dim, out_dim, e_dim):
        super(GATLayer, self).__init__()
        self.w1 = nn.Linear(in_dim + e_dim, out_dim, bias=False)
        self.w2 = nn.Linear(in_dim + out_dim, out_dim, bias=False)
        self.w_att = nn.Linear(in_dim + out_dim, 1, bias=False)
        self.reset_parameters()
    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.w1.weight, gain=gain)
        nn.init.xavier_normal_(self.w2.weight, gain=gain)
        nn.init.xavier_normal_(self.w_att.weight, gain=gain)
    def edge_attention(self, edges):
        z = torch.cat([edges.src['h'], edges.data['m']], -1)
        a = self.w_att(z)
        alpha = F.leaky_relu(a)
        return {'e': alpha}
    def msg1(self, edges):
        return {'m': self.w1(th.cat([edges.src['h'], edges.data['h']], -1))}
    
    def message_func(self, edges):
        return {'z': edges.data['m'], 'e': edges.data['e']}
    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1) # 归一化每一条入边的注意力系数
        h = torch.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h_neigh': h}
    def forward(self, g_dgl, hfeat, efeat):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = hfeat
            g.edata['h'] = efeat
            g.apply_edges(self.msg1)
            g.apply_edges(self.edge_attention) # 为每一条边获得其注意力系数
            g.update_all(self.message_func, self.reduce_func)
            g.ndata['h'] = F.relu(self.w2(th.cat([g.ndata['h'], g.ndata['h_neigh']], -1)))
            return g.ndata['h']
class Model(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, e_dim):
        super().__init__()
        self.layer1 = GATLayer(in_dim, hidden_dim, e_dim)
        self.layer2 = GATLayer(hidden_dim, out_dim, e_dim)
        self.pred = MLPPredictor(out_dim, 10)
    def forward(self, g, nfeats, efeats):
        nfeats = self.layer1(g, nfeats, efeats)
        nfeats = self.layer2(g, nfeats, efeats)
        return self.pred(g, nfeats)
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)
    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(th.cat([h_u, h_v], 1))
        return {'score': score}
    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [4]:
G = load_graphs("./ToN-loT/tonlot_train_data.bin")[0][0]

In [5]:
G.ndata['h'] = th.ones(G.num_nodes(), G.edata['h'].shape[1]) 
G.edata['train_mask'] = th.ones(len(G.edata['h']), dtype= th.bool)

In [6]:
G

Graph(num_nodes=139851, num_edges=645460,
      ndata_schemes={'h': Scheme(shape=(38,), dtype=torch.float32)}
      edata_schemes={'type': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(38,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.bool)})

In [7]:
Counter(G.edata['type'].cpu().numpy())

Counter({0: 420000,
         5: 28000,
         4: 28000,
         2: 28000,
         1: 28000,
         3: 28000,
         6: 28000,
         7: 28000,
         8: 28000,
         9: 1460})

In [8]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(G.edata['type'].cpu().numpy()),
                                                 G.edata['type'].cpu().numpy())
class_weights = th.FloatTensor(class_weights)

/home/xiaoyujie/anaconda3/envs/densegat2/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4 5 6 7 8 9], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [9]:
device = 'cuda:1'

In [10]:
G = G.to(device)

In [11]:
class_weights = class_weights.to(device)

In [12]:
node_features = G.ndata['h']
edge_features = G.edata['h']

edge_label = G.edata['type']
train_mask = G.edata['train_mask']

In [13]:
in_dim = node_features.size(1)
hidden_dim = 20
out_dim = 10
e_dim = in_dim

In [14]:
model = Model(in_dim, hidden_dim, out_dim, e_dim)

In [15]:
opt = th.optim.Adam(model.parameters())

In [16]:
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [17]:
import time

In [18]:
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

In [34]:
model = model.to(device)
for epoch in range(1,15000):
    pred = model(G, node_features, edge_features)
    loss = criterion(pred[train_mask], edge_label[train_mask])
    opt.zero_grad()
    loss.backward()
    opt.step()
    if epoch % 100 == 0:
        print('loss',loss.item(),'Epoch:', epoch ,' Training acc:', compute_accuracy(pred[train_mask], edge_label[train_mask]))
        torch.save(model.state_dict(), 'egat_tonlot_model' + str(epoch))

loss 0.4237349331378937 Epoch: 100  Training acc: 0.896831750869751
loss 0.41941747069358826 Epoch: 200  Training acc: 0.8984026908874512
loss 0.41663455963134766 Epoch: 300  Training acc: 0.8994515538215637
loss 0.4133221209049225 Epoch: 400  Training acc: 0.9001332521438599
loss 0.4100708067417145 Epoch: 500  Training acc: 0.9004958271980286
loss 0.4078136086463928 Epoch: 600  Training acc: 0.9007126688957214
loss 0.4063272178173065 Epoch: 700  Training acc: 0.9020915627479553
loss 0.4032445549964905 Epoch: 800  Training acc: 0.9022774696350098


KeyboardInterrupt: 

In [35]:
model = model.to('cpu')
cm, cr = test(G_test, model)
print(cm),    

[[161693   3549    746      2    694    120   2912   1872   4595   3817]
 [    45  10911    700      1     10    229      0     38     12     54]
 [   145    162  10569      4      6    864     48      0     35    167]
 [  2672     20    114    289    771   5558    373      0      0   2203]
 [   915      1    402     72   3215   6605    571      0      2    217]
 [  1776    192   1284    314    721   7153    117      8      6    429]
 [  1705     38    803     83    194   1761   7205      0      0    211]
 [    26    270     38      0      0      0      0   9882   1712     72]
 [    25      2     12      0      0      0      0    118  11807     36]
 [    12      8      0      8      2    340     14      0      0    242]]


(None,)

In [36]:
print(cr)

              precision    recall  f1-score   support

           0     0.9567    0.8983    0.9266    180000
           1     0.7201    0.9093    0.8037     12000
           2     0.7205    0.8808    0.7926     12000
           3     0.3739    0.0241    0.0453     12000
           4     0.5728    0.2679    0.3651     12000
           5     0.3161    0.5961    0.4131     12000
           6     0.6410    0.6004    0.6201     12000
           7     0.8292    0.8235    0.8263     12000
           8     0.6498    0.9839    0.7827     12000
           9     0.0325    0.3866    0.0599       626

    accuracy                         0.8060    276626
   macro avg     0.5813    0.6371    0.5635    276626
weighted avg     0.8318    0.8060    0.8047    276626



In [68]:
print(cr)

              precision    recall  f1-score   support

           0     0.9273    0.9586    0.9427    180000
           1     0.6239    0.8973    0.7360     12000
           2     0.8775    0.9015    0.8893     12000
           3     0.4882    0.0378    0.0702     12000
           4     0.5498    0.6542    0.5975     12000
           5     0.3986    0.6476    0.4935     12000
           6     0.5955    0.6318    0.6131     12000
           7     0.8183    0.7367    0.7753     12000
           8     0.1317    0.0270    0.0448     12000
           9     0.3883    0.1693    0.2358       626

    accuracy                         0.8208    276626
   macro avg     0.5799    0.5662    0.5398    276626
weighted avg     0.7988    0.8208    0.7970    276626



In [20]:
def test(G_test, model):
    test_node_features = G_test.ndata['h']
    test_edge_features = G_test.edata['h']
    y_true = G_test.edata['type'].detach().numpy()
    pred = model(G_test, test_node_features, test_edge_features)
    y_pred = pred.detach().numpy()
    y_pred = np.argmax(y_pred, -1)
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred, digits=4)
    return cm, cr

In [21]:
G_test = load_graphs("./ToN-loT/tonlot_test_data.bin")[0][0]

In [24]:
G_test

Graph(num_nodes=78288, num_edges=276626,
      ndata_schemes={}
      edata_schemes={'type': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(38,), dtype=torch.float32)})

In [22]:
Counter(G.edata['type'].cpu().numpy())

Counter({0: 420000,
         5: 28000,
         4: 28000,
         2: 28000,
         1: 28000,
         3: 28000,
         6: 28000,
         7: 28000,
         8: 28000,
         9: 1460})

In [32]:
Counter(G_test.edata['label'].numpy())

Counter({0: 180000, 1: 96626})

In [25]:
G_test.ndata['h'] = th.ones(G_test.num_nodes(), G_test.edata['h'].shape[1]) 

In [23]:
Counter(G_test.edata['label'].numpy())

Counter({1: 99016, 0: 115598, 3: 5464, 2: 286, 4: 48})

In [ ]:
class GATLayer(nn.Module):
    def __init__(self, in_dim , out_dim):
        super(GATLayer, self).__init__()
        self.W = nn.Linear(in_dim, out_dim, bias=False)
        self.attn_fc = nn.Linear(3 * out_dim, 1, bias=False)
        self.reset_parameters()
    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.W.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_fc.weight, gain=gain)
    def edge_attention(self, edges):
        z = torch.cat([edges.src['z'], edges.dst['z'], edges.data['z']], -1)
        a = self.attn_fc(z)
        alpha = F.leaky_relu(a)
        return {'e': alpha}
    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e'] ,'m': edges.data['z']}
    def reduce_func(self, nodes):
        alpha = F.softmax(nodes.mailbox['e'], dim=1) # 归一化每一条入边的注意力系数
        h = torch.sum(alpha * nodes.mailbox['m'], dim=1)
        return {'h': F.relu(h)}
    def forward(self, g_dgl, hfeat, efeat):
        with g_dgl.local_scope():
            g = g_dgl
            z1 = self.W(hfeat)
            z2 = self.W(efeat)
            g.ndata['z'] = z1 # 每个节点的特征
            g.edata['z'] = z2 # 每条边的特征
            g.apply_edges(self.edge_attention) # 为每一条边获得其注意力系数
            g.update_all(self.message_func, self.reduce_func)
            return g.ndata['h'], g.edata['z']
class Model(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.layer1 = GATLayer(in_dim, hidden_dim)
        self.layer2 = GATLayer(hidden_dim, out_dim)
        self.pred = MLPPredictor(out_dim, 5)
    def forward(self, g, nfeats, efeats):
        nfeats, efeats = self.layer1(g, nfeats, efeats)
#         nfeats, efeats = self.layer2(g, nfeats, efeats)
        return self.pred(g, nfeats)
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)
    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(th.cat([h_u, h_v], 1))
        return {'score': score}
    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']